# NFT/Pre-NFT Inter-annotator Agreement Analysis

In [ ]:
# Imports
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from itertools import combinations
import cv2 as cv
from tqdm.notebook import tqdm
from colorama import Fore, Style
from IPython.display import clear_output
from multiprocessing import Pool
import re

import pandas as pd
from pandas import read_csv, DataFrame

from os import makedirs
from os.path import join

from nft_helpers.utils import load_yaml, imread, imwrite
from nft_helpers.plot import format_plot_edges
from nft_helpers.box_and_contours import line_to_xys, xys_to_line
from nft_helpers.roi_utils import read_roi_txt_file

# Parameters
cf = load_yaml()
COLORS = [f'#{c}' for c in cf.colors]
save_dir = join(cf.datadir, 'results/inter-annotator-agreement')
makedirs(save_dir, exist_ok=True)

## Annotation Counts

In [ ]:
# Bar plot of counts.
df = read_csv('csvs/annotations.csv')
df = df[(df.cohort == 'Annotated-Cohort') & (df.roi_group == 'ROIv2')]

# Format the inter-annotator annotations for plotting as bars.
ann_bar_df = []

for annotator in df.annotator.unique():
    ann_df = df[df.annotator == annotator]
    
    # Add the sum of all
    ann_bar_df.append([annotator, 'Total', len(ann_df)])
    
    for ann_type, counts in ann_df.label.value_counts().items():
        ann_bar_df.append([annotator, ann_type, counts])
                
ann_bar_df = DataFrame(
    ann_bar_df, columns=['annotator', 'label', 'counts']
).sort_values(by=['annotator', 'label'], ascending=[True, False])

kwargs = {'edgecolor': 'k', 'lw': 3}
colors = [COLORS[2], COLORS[1], COLORS[0]]
ax = sns.barplot(data=ann_bar_df, x='annotator', y='counts', hue='label', 
                 palette=colors, hue_order=['Total', 'iNFT', 'Pre-NFT'],
                 **kwargs)
for i, bars in enumerate(ax.containers):
    for bar in bars:
        bar.set_hatch(cf.hatches[2-i])
        
plt.legend(bbox_to_anchor=(1.30, 1))
plt.setp(ax.get_legend().get_texts(), fontsize='16') 

ax = format_plot_edges(ax)
ax.tick_params(axis='both', which='both', direction='out', length=10, width=3)
plt.xticks(rotation=90, fontsize=18)
plt.xlabel(None)
plt.yticks(fontsize=18)
plt.ylabel('Number of Annotations', fontsize=18)
plt.legend(ncol=3, fontsize=14, bbox_to_anchor=(0.5, 1.10), loc='upper center')
# plt.title('Annotations for Inter-annotator\nAgreement Analysis', fontsize=18)
plt.savefig(join(save_dir, 'annotation-counts.png'), bbox_inches='tight', 
            dpi=300)
plt.show()

ann_bar_df.to_csv(join(save_dir, 'annotation-counts.csv'), index=False)
display(ann_bar_df.head())

## Cohen's Kappas
For each annotator create 30 ROI masks (1 per label for 15 ROIs).

In [ ]:
# Compile annotations into a dataframe for easy comparison between annotators.
anns = []

df = read_csv('csvs/matching-annotations.csv')
annotators = sorted(df.iloc[0].groups.split(';'))

for _, r in df.iterrows():
    labels = r.labels.split(';')
    groups = r.groups.split(';')
    labels = {ann: lb for lb, ann in zip(labels, groups)}
    
    box = (line_to_xys(r.box_coords) - [r.roi_im_left, r.roi_im_top]).astype(int)
    x1, y1 = np.min(box, axis=0)
    x2, y2 = np.max(box, axis=0)
    
    row = [
        r.wsi_name, r.region, r.roi_im_path, r.roi_im_left, r.roi_im_top,
        r.roi_im_right - r.roi_im_left, r.roi_im_bottom - r.roi_im_top,
        xys_to_line(line_to_xys(r.roi_corners) - [r.roi_im_left, r.roi_im_top]),
        (x1, y1), (x2, y2),
    ]
    
    for ann in annotators:
        label = labels[ann]
        
        if label == 'iNFT':
            row.append(2)
        elif label == 'Pre-NFT':
            row.append(1)
        else:
            row.append(0)
        
    anns.append(row)
    
anns = DataFrame(
    anns, 
    columns=['wsi_name', 'region', 'roi_fp', 'roi_x1', 'roi_y1', 'roi_w', 
             'roi_h', 'roi_corners', 'pt1', 'pt2'] + annotators
)
display(anns.head())

In [ ]:
# Calculate the confusion matrix between pair of annotators.
cm_dir = join(save_dir, 'cms')
makedirs(cm_dir, exist_ok=True)
labels = ('Background', 'Pre-NFT', 'iNFT')

for pair in combinations(annotators, 2):
    # Remove annotations that both these pairs labeled as background.
    pair_df = anns[list(pair)]
    pair_df = pair_df[
        (pair_df[pair[0]] != 0) | (pair_df[pair[1]] != 0)
    ]
    
    # Calculate the confusion matrix.
    cm = confusion_matrix(pair_df[pair[0]], pair_df[pair[1]])
    ncm = confusion_matrix(pair_df[pair[0]], pair_df[pair[1]], normalize='all')
    # Plot the confusion matrix
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.matshow(cm, cmap=plt.cm.Blues)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            s = '' if i == j == 0 else cm[i, j]
            ax.text(x=j, y=i,s=s, va='center', ha='center', size=18, 
                    color='k', weight='bold')
    plt.gca().xaxis.tick_bottom()
    plt.xticks(np.arange(0, 3), labels, fontsize=18)
    plt.yticks(np.arange(0, 3), labels, fontsize=18)
    plt.ylabel(pair[0], fontsize=18)
    plt.xlabel(pair[1], fontsize=18)
    plt.title('Confusion Matrix', fontsize=18)
    plt.savefig(join(cm_dir, f'{pair[0]}-{pair[1]}-cm.png'), bbox_inches='tight', 
                dpi=300)
    plt.show()

In [ ]:
# Cohen's kappa for annotation masks (parallel processing)
roi_kappas = []
labels = ['Pre-NFT', 'iNFT']
pairs = list(combinations(annotators, 2))
wsi_names = anns.wsi_name.unique()
resize_factor = 0.5

# For each WSI.
params = []

for n, wsi_name in enumerate(wsi_names):
    for pair in pairs:
        pair = sorted(list(pair))
        params.append([wsi_name, pair[0], pair[1]])

        
def masks_cohens_kappa(wsi_name: str, annotator1: str, annotator2: str):
    """Calculate the Cohen's kappa between two masks for annotators.
    
    Args:
        wsi_name: WSI name => unique ROI since only one ROI per WSI.
        annotator1: Annotator 1.
        annotator2: Annotator 2.
        
    """
    data = []
    
    roi_df = anns[anns.wsi_name == wsi_name]
    
    # Create a binary mask of the ROI region
    r = roi_df.iloc[0]
    roi_mask = np.zeros((r.roi_h, r.roi_w), dtype=np.uint8)
    roi_mask = cv.drawContours(roi_mask, [line_to_xys(r.roi_corners)], -1, 255, 
                               cv.FILLED)
    
    # resize for quicker computations
    roi_mask = cv.resize(roi_mask, None, fx=resize_factor, fy=resize_factor
                        ).flatten()
    
    # Loop through each class
    for label in [1, 2]:
        # Create a label mask for this label for each annotator.
        df = roi_df[roi_df[annotator1] == label]
        mask1 = np.zeros((r.roi_h, r.roi_w), dtype=np.uint8)
            
        for _, r in df.iterrows():
            mask1 = cv.rectangle(mask1, r.pt1, r.pt2, 1, cv.FILLED)
                
        # Resize and filter by the pixels inside the ROI.
        mask1 = cv.resize(mask1, None, fx=resize_factor, fy=resize_factor
                         ).flatten()[roi_mask == 255]
        
        df = roi_df[roi_df[annotator2] == label]
        mask2 = np.zeros((r.roi_h, r.roi_w), dtype=np.uint8)
            
        for _, r in df.iterrows():
            mask2 = cv.rectangle(mask2, r.pt1, r.pt2, 1, cv.FILLED)
                
        # Resize and filter by the pixels inside the ROI.
        mask2 = cv.resize(mask2, None, fx=resize_factor, fy=resize_factor
                         ).flatten()[roi_mask == 255]
        
        # Cohen's kappa
        if np.count_nonzero(mask1) + np.count_nonzero(mask2):
            k = cohen_kappa_score(mask1, mask2)
        else:
            # Kappa does not exist when both masks are all 0s  or all 1s. This
            # is perfect agreement.
            k = 1.
                              
        data.append([f'{annotator1}-{annotator2}', annotator1, annotator2, 
                     label-1, wsi_name, k])
        
    return data
        
    
with Pool(20) as pool:
    jobs = [pool.apply_async(func=masks_cohens_kappa, args=(p[0], p[1], p[2])) 
            for p in params]
        
    roi_df = []
    
    for job in tqdm(jobs):
        roi_df.extend(job.get())
    
roi_df = DataFrame(roi_df, columns=['key', 'annotator1', 'annotator2', 'label',
                                    'wsi_name', 'k'])
roi_df.to_csv(join(save_dir, 'cohens-kappas.csv'), index=False)
roi_df.head()

## Heatmaps of Cohen's Kappa

In [ ]:
# Plot heatmap of pair kappas between anntotation masks.
def plot_tri_heatmap(data: np.array, labels: list, figsize: (int, int) = (7,7), 
                     title: str = None, save_fp: str = None, **kwargs: dict
                    ) -> np.array:
    """Create a correlation heatmap from an array, only plotting the bottom half
    triangle of the heatmap. 
    
    Args:
        data: Data to plot, the data is assumed to be symetrical and only the 
            bottom left triangle of the heatmap will be shown.
        labels: Labels on both axis, ordered from top to bottom and left to 
            right.
        figsize: (width, height) of figure.
        title: Title of figure.
        save_fp: Filepath to save figure to.
        kwargs: Keyword arguments to pass to seaborn.heatmap()
       
    Returns:
        The input data array.
        
    """
    fig, ax = plt.subplots(figsize=figsize)
    mask = np.triu(np.ones_like(data), k=1)
    ax = sns.heatmap(data, annot=True, mask=mask, xticklabels=labels, 
                     yticklabels=labels, ax=ax, **kwargs)
    ax.set_xticks(ax.get_xticks(), labels, size=16)
    ax.set_yticks(ax.get_yticks(), labels, size=16, rotation=360)
    ax.set_facecolor('k')
    
    if title is not None:
        plt.title(title, fontsize=18, weight='bold')
        
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=16)
        
    return ax


# Labels
roi_df = read_csv('/workspace/data/results/inter-annotator-agreement/cohens-kappas.csv')

annotators = set(roi_df.annotator1.unique())
annotators.update(set(roi_df.annotator2.unique()))
annotators = sorted(list(annotators))

results = "Average Cohen's Kappa for Annotations\n"
results += '-' * (len(results)-1) + '\n\n'

for label in [0, 1]:
    kappa_hm = np.zeros((len(annotators), len(annotators)))
    
    expert_expert = {}
    novice_novice = {}
    expert_novice = {}
    
    for i, ann1 in enumerate(annotators):
        for j, ann2 in enumerate(annotators):
            if ann1 == ann2:
                kappa_hm[i, j] = 1
            else:
                kappa_hm[i, j] = 0
                anns = sorted([ann1, ann2])
                key = f'{anns[0]}-{anns[1]}'
                
                k = roi_df[
                    (roi_df.key == key) & (roi_df.label == label)
                ].k.mean()
                
                if re.search('expert\d-expert\d', key) and key not in expert_expert:
                    expert_expert[key] = k
                elif re.search('expert\d-novice\d', key) and key not in expert_novice:
                    expert_novice[key] = k
                elif re.search('novice\d-novice\d', key) and key not in novice_novice:
                    novice_novice[key] = k

                kappa_hm[i, j] = k
                                
    kwargs = {'cmap': 'viridis', 'annot_kws': {"size":16}, 
              'linecolor': 'w', 'linewidths': 0}
    
    # Calculate the average and standard deviation of the pair kappas.
    mask = np.triu(np.ones_like(kappa_hm))
    kappas = kappa_hm[mask == 0]

    ax = plot_tri_heatmap(
        kappa_hm, 
        ['E1', 'E2', 'E3', 'E4', 'E5', 'N1', 'N2', 'N3', 'N4'], 
        figsize=(10,10), 
        title=f"{cf.labels[label]} ({np.mean(kappas):.2f} " + u"\u00B1" + \
              f' {np.std(kappas):.2f})',
        **kwargs
    )
    plt.savefig(join(save_dir, f'{cf.labels[label]}-hm.png'), 
                bbox_inches='tight', dpi=300)
    plt.show()

    expert_expert = list(expert_expert.values())
    novice_novice = list(novice_novice.values())
    expert_novice = list(expert_novice.values())
    all_pairs = expert_expert + novice_novice + expert_novice

    results += f'{cf.labels[label]}:\n'
    results += f'   - Expert vs Expert: {np.mean(expert_expert):.2f} ' + \
               u"\u00B1" + f' {np.std(expert_expert):.2f}\n'
    results += f'   - Expert vs Novice: {np.mean(expert_novice):.2f} ' + \
               u"\u00B1" + f' {np.std(expert_novice):.2f}\n'
    results += f'   - Novice vs Novice: {np.mean(novice_novice):.2f} ' + \
               u"\u00B1" + f' {np.std(novice_novice):.2f}\n'
    results += f'   - All pairs: {np.mean(all_pairs):.2f} ' + \
               u"\u00B1" + f' {np.std(all_pairs):.2f}\n\n'
    
print(results.strip())

with open(join(save_dir, 'results.txt'), 'w') as fh:
    fh.write(results.strip())

## Supplementary Figures

### Point annotations to Class Binary Masks

In [ ]:
# Get an ROI and draw the point annotations.
rois_df = read_csv('csvs/labeled-rois.csv')
r = rois_df.iloc[0]

# Read the RGB image of ROI.
img = imread(r.roi_im_path)
mask1 = np.zeros(img.shape, dtype=np.uint8)
mask2 = np.zeros(img.shape, dtype=np.uint8)

# Draw the point annotations.
points = read_roi_txt_file(r.roi_labels)

for pt in read_roi_txt_file(r.roi_labels):
    lb, x1, y1, x2, y2 = pt
    
    xc, yc = int((x1 + x2) / 2), int((y1 + y2) / 2)
    
    img = cv.circle(img, (xc, yc), 45, (255, 0, 0) if lb else (0, 0, 255), 10)
    
    if lb:
        mask1 = cv.rectangle(mask1, (x1, y1), (x2, y2), (255, 0, 0), cv.FILLED)
    else:
        mask2 = cv.rectangle(mask2, (x1, y1), (x2, y2), (0, 0, 255), cv.FILLED)
    
plt.figure(figsize=(5,10))
plt.imshow(img)
plt.axis('off')
plt.savefig(join(save_dir, f'sample-roi-with-points.png'), bbox_inches='tight',
            dpi=300)
plt.close()

plt.figure(figsize=(5,10))
plt.imshow(mask1)
plt.axis('off')
plt.savefig(join(save_dir, f'sample-iNFT-label-mask.png'), bbox_inches='tight',
            dpi=300)
plt.close()

plt.figure(figsize=(5,10))
plt.imshow(mask2)
plt.axis('off')
plt.savefig(join(save_dir, f'sample-Pre-NFT-label-mask.png'), 
            bbox_inches='tight', dpi=300)
plt.close()